# 机器学习练习 4 - 神经网络

本章代码涵盖了基于Python的解决方案，用于Coursera机器学习课程的第四个编程练习。 请参考[练习文本](ex4.pdf)了解详细的说明和公式。


对于这个练习，我们将再次处理手写数字数据集，这次使用反向传播的前馈神经网络。 我们将通过反向传播算法实现神经网络成本函数和梯度计算的非正则化和正则化版本。 我们还将实现随机权重初始化和使用网络进行预测的方法。

由于我们在练习3中使用的数据集是相同的，所以我们将重新使用代码来加载数据。

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [7]:
data = loadmat("ex4data1.mat")
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

由于我们以后需要这些（并将经常使用它们），我们先来创建一些有用的变量。

In [8]:
X = data["X"]
y = data["y"]

X.shape, y.shape  # 看下维度

((5000, 400), (5000, 1))

我们也需要对我们的y标签进行一次one-hot 编码。 one-hot 编码将类标签n（k类）转换为长度为k的向量，其中索引n为“hot”（1），而其余为0。 Scikitlearn有一个内置的实用程序，我们可以使用这个。

In [9]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)
# sparse_output=False，可以确保输出是一个密集的numpy数组

y_onehot = encoder.fit_transform(y)
# 首先使用y数据集（假设y是一个一维数组或列向量，包含了分类标签）来拟合encoder，然后对同一数据执行转换。fit_transform()方法既根据y的数据学习类别信息（如果类别在训练时未知），又直接对y进行编码转换。转换后的结果y_onehot是一个二维数组，其中每一行代表原始y中的一个样本，每一列对应一个类别，如果某个样本属于该类别，则该列的值为1，其余为0。

y_onehot.shape

(5000, 10)

In [10]:
y[0], y_onehot[0, :]

(array([10], dtype=uint8), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]))

我们要为此练习构建的神经网络具有与我们的实例数据（400 +偏置单元）大小匹配的输入层，25个单位的隐藏层（带有偏置单元的26个），以及一个输出层， 10个单位对应我们的一个one-hot编码类标签。 有关网络架构的更多详细信息和图像，请参阅“练习”文件夹中的PDF。

我们需要实现的第一件事是评估一组给定的网络参数的损失的代价函数。 源函数在练习文本中（看起来很吓人）。 以下是代价函数的代码。

# sigmoid 函数
$g$ 代表一个常用的逻辑函数（logistic function），即S形函数（Sigmoid function），其公式为： 

$$g(z) = \frac{1}{1 + e^{-z}}$$

合起来，我们得到逻辑回归模型的假设函数： 

$$h_\theta(x) = \frac{1}{1 + e^{-\theta^T X}}$$

# 前向传播函数
> (400 + 1) -> (25 + 1) -> (10)

<img style="float: left;" src="../img/nn_model.png">

In [15]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [11]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]

    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    # 使用np.insert函数在X的第一列插入一个全1向量，创建a1，形状为(m, n+2)。这相当于在输入层每个节点处添加了一个偏置单元。
    
    z2 = a1 * theta1.T
    # 计算z2，即隐藏层节点的线性输入，形状为(m, j)。这通过将a1与theta1转置后的乘积得到。
    
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    # 对z2应用sigmoid激活函数，得到a2。由于Sigmoid函数的输入和输出都是 (0,1) 之间的值，a2的形状也是(m, j)。同样，在a2的第一列插入一个全1向量，以在隐藏层的每个节点处添加偏置单元。
    
    z3 = a2 * theta2.T
    h = sigmoid(z3)
    # 对z3应用sigmoid激活函数，得到h，形状为(m, k)。h是模型的预测输出，对应于每个样本的输出节点的激活值。

    return a1, z2, a2, z3, h

# 代价函数
<img style="float: left;" src="../img/nn_cost.png">

交叉熵损失（Cross-Entropy Loss）是分类问题中常用的一种损失函数，特别是在神经网络和逻辑回归中。它衡量了模型预测的概率分布与真实标签之间的差异。在二分类问题中，交叉熵损失可以表示为：

如果第i个样本的真实标签是$y_i$（0或1），模型预测的概率是$h_i$，则单个样本的交叉熵损失$L_i$计算如下：

 $$L_i = -y_i \log(h_i) - (1 - y_i) \log(1 - h_i)$$

这里的$log$是自然对数。当$y_i$为1时，我们关注的是$log(h_i)$，因为这是模型正确预测为正类的概率；当$y_i$为0时，我们关注的是$log(1 - h_i)$，这是模型正确预测为负类的概率。

In [12]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
# params：一个一维数组，包含了所有层的权重参数。
# input_size：输入层的特征数量。
# hidden_size：隐藏层的节点数量。
# num_labels：输出层的类别数量。
# X：形状为(m, n)的二维数组，其中m是样本数量，n是输入层的特征数。输入数据。
# y：形状为(m, k)的二维数组，其中k是输出层的类别数。实际的标签，通常是一系列的二进制值。
# learning_rate：学习率，用于梯度下降优化。

    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array into parameter matrices for each layer
    # 从params数组中重塑权重参数，将其分为theta1（输入层到隐藏层）和theta2（隐藏层到输出层）。reshape函数用于调整数组形状，np.matrix用于转换为矩阵形式。
    
    theta1 = np.matrix(
        np.reshape(
            params[: hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))
        )
    )
    theta2 = np.matrix(
        np.reshape(
            params[hidden_size * (input_size + 1) :], (num_labels, (hidden_size + 1))
        )
    )

    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

    # compute the cost
    J = 0
    
    # 遍历每一个样本，计算交叉熵损失。对于第i个样本，计算两部分：
    # 第一部分：first_term，对应于正确类别的负对数似然。
    # 第二部分：second_term，对应于非正确类别的负对数似然。
    # 对这两部分求和，然后除以样本总数m，得到平均代价J。
    for i in range(m):
        first_term = np.multiply(-y[i, :], np.log(h[i, :]))
        # 这部分计算的是当y为1时的损失。np.multiply按元素相乘，np.log(h[i, :])计算的是模型预测概率的对数.
        
        second_term = np.multiply((1 - y[i, :]), np.log(1 - h[i, :]))
        #  这部分计算的是当y为0时的损失。
        
        J += np.sum(first_term - second_term)

    J = J / m

    return J

这个Sigmoid函数我们以前使用过。 前向传播函数计算给定当前参数的每个训练实例的假设。 它的输出形状应该与y的一个one-hot编码相同。 

In [13]:
# 这段代码是神经网络模型训练前的准备工作，确保模型的权重参数被随机初始化，然后准备好进行前向传播和反向传播来更新这些权重。

# 初始化设置
input_size = 400
# 输入层的特征数量

hidden_size = 25
# 隐藏层的节点数量

num_labels = 10
# 输出层的类别数量

learning_rate = 1

# 随机初始化完整网络参数大小的参数数组
# 创建一个一维数组params，大小等于隐藏层和输出层的所有权重参数的总和。
params = (
    np.random.random(
        size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)
    )
    - 0.5
) * 0.25

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

# 将参数数组解开为每个层的参数矩阵
theta1 = np.matrix(
    np.reshape(
        params[: hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))
    )
)
# 额外的1是用于添加偏置单元。

theta2 = np.matrix(
    np.reshape(
        params[hidden_size * (input_size + 1) :], (num_labels, (hidden_size + 1))
    )
)

theta1.shape, theta2.shape

((25, 401), (10, 26))

In [16]:
a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((5000, 401), (5000, 25), (5000, 26), (5000, 10), (5000, 10))

代价函数在计算假设矩阵h之后，应用代价函数来计算y和h之间的总误差。

In [17]:
cost(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)

6.781753623623506

# 正则化代价函数
我们的下一步是增加代价函数的正则化。  它实际上并不像看起来那么复杂 - 事实上，正则化术语只是我们已经计算出的代价的一个补充。 下面是修改后的代价函数。
<img style="float: left;" src="../img/nn_regcost.png">

In [18]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(
        np.reshape(
            params[: hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))
        )
    )
    theta2 = np.matrix(
        np.reshape(
            params[hidden_size * (input_size + 1) :], (num_labels, (hidden_size + 1))
        )
    )

    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i, :], np.log(h[i, :]))
        second_term = np.multiply((1 - y[i, :]), np.log(1 - h[i, :]))
        J += np.sum(first_term - second_term)

    J = J / m

    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (
        np.sum(np.power(theta1[:, 1:], 2)) + np.sum(np.power(theta2[:, 1:], 2))
    )

    return J

In [19]:
cost(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)

6.787124161949125

接下来是反向传播算法。 反向传播参数更新计算将减少训练数据上的网络误差。 我们需要的第一件事是计算我们之前创建的Sigmoid函数的梯度的函数。

由于 
$$ \sigma(z) = \frac{1}{1 + e^{-z}} $$
因此
$$  \sigma'(z) = \sigma(z)(1 - \sigma(z)) $$

In [20]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

现在我们准备好实施反向传播来计算梯度。 由于反向传播所需的计算是代价函数中所需的计算过程，我们实际上将扩展代价函数以执行反向传播并返回代价和梯度。

In [22]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(
        np.reshape(
            params[: hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))
        )
    )
    theta2 = np.matrix(
        np.reshape(
            params[hidden_size * (input_size + 1) :], (num_labels, (hidden_size + 1))
        )
    )

    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    # h是预测输出

    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)

    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i, :], np.log(h[i, :]))
        second_term = np.multiply((1 - y[i, :]), np.log(1 - h[i, :]))
        J += np.sum(first_term - second_term)

    J = J / m

    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (
        np.sum(np.power(theta1[:, 1:], 2)) + np.sum(np.power(theta2[:, 1:], 2))
    )

    # perform backpropagation
    for t in range(m):
        # 分别提取第t个样本的a1, z2, a2, ht, yt，这些是前向传播过程中对应的激活值和输出。
        a1t = a1[t, :]  # (1, 401)
        z2t = z2[t, :]  # (1, 25)
        a2t = a2[t, :]  # (1, 26)
        ht = h[t, :]  # (1, 10)
        yt = y[t, :]  # (1, 10)

        d3t = ht - yt  # (1, 10) 计算输出层的误差，即预测输出ht与真实标签yt之差。

        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        # 为了计算隐藏层误差d2t，首先在z2t（隐藏层的线性输出）前插入一个1（偏置项的误差），使其与权重矩阵维度匹配。
        
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        # 然后，根据链式法则计算d2t，即隐藏层误差，涉及到权重矩阵theta2的转置与输出层误差d3t的乘积，之后应用sigmoid的导数作用于扩展后的z2t。

        delta1 = delta1 + (d2t[:, 1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        # delta1和delta2分别累加隐藏层到输入层和输出层到隐藏层的权重梯度。
        # delta1累加了d2t（不包括偏置项的误差）与当前样本的输入激活a1t的乘积。
        # delta2累加了d3t与隐藏层激活a2t的乘积。

    # 得到平均梯度。
    delta1 = delta1 / m
    delta2 = delta2 / m

    # unravel the gradient matrices into a single array
    # 最后，使用np.concatenate将两个梯度矩阵delta1和delta2展平为一维数组，并合并成一个大的梯度数组grad。
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))

    return J, grad

反向传播计算的最难的部分（除了理解为什么我们正在做所有这些计算）是获得正确矩阵维度。 顺便说一下，你容易混淆了A * B与np.multiply（A，B）使用。 基本上前者是矩阵乘法，后者是元素乘法（除非A或B是标量值，在这种情况下没关系）。
无论如何，让我们测试一下，以确保函数返回我们期望的。

In [23]:
J, grad = backprop(
    params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate
)
J, grad.shape

(6.787124161949125, (10285,))

我们还需要对反向传播函数进行一个修改，即将梯度计算加正则化。 最后的正式版本如下。

In [28]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(
        np.reshape(
            params[: hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))
        )
    )
    theta2 = np.matrix(
        np.reshape(
            params[hidden_size * (input_size + 1) :], (num_labels, (hidden_size + 1))
        )
    )

    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)

    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i, :], np.log(h[i, :]))
        second_term = np.multiply((1 - y[i, :]), np.log(1 - h[i, :]))
        J += np.sum(first_term - second_term)

    J = J / m

    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (
        np.sum(np.power(theta1[:, 1:], 2)) + np.sum(np.power(theta2[:, 1:], 2))
    )

    # perform backpropagation
    for t in range(m):
        a1t = a1[t, :]  # (1, 401)
        z2t = z2[t, :]  # (1, 25)
        a2t = a2[t, :]  # (1, 26)
        ht = h[t, :]  # (1, 10)
        yt = y[t, :]  # (1, 10)

        d3t = ht - yt  # (1, 10)

        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        # d2t = np.multiply((d3t, theta2), sigmoid_gradient(z2t))  # (1, 26)
        d2t = np.multiply((theta2.T @ d3t.T).T, sigmoid_gradient(z2t))  # 注意这里使用了矩阵乘法 @ 

        delta1 = delta1 + (d2t[:, 1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t

    delta1 = delta1 / m
    delta2 = delta2 / m

    # add the gradient regularization term
    delta1[:, 1:] = delta1[:, 1:] + (theta1[:, 1:] * learning_rate) / m
    delta2[:, 1:] = delta2[:, 1:] + (theta2[:, 1:] * learning_rate) / m

    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))

    return J, grad

In [29]:
J, grad = backprop(
    params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate
)
J, grad.shape

(6.787124161949125, (10285,))

我们终于准备好训练我们的网络，并用它进行预测。 这与以往的具有多类逻辑回归的练习大致相似。

In [31]:
from scipy.optimize import minimize
# scipy.optimize.minimize函数来执行一个优化任务，目的是最小化一个目标函数（在这里是神经网络的损失函数，通过backprop函数实现）

# minimize the objective function
fmin = minimize(
    fun=backprop, # 待优化的目标函数
    x0=params, # 初始猜测值，即优化开始时权重参数的初始值。在这里，params是之前随机初始化的权重参数数组。
    args=(input_size, hidden_size, num_labels, X, y_onehot, learning_rate),
    method="TNC",
    # 优化算法的选择。"TNC"代表“Truncated Newton (TNC)”，这是一种混合算法，结合了牛顿法和拟牛顿法的优点，适用于大规模问题。它能在每次迭代中解决一个近似的牛顿方程，并且可以处理带约束的问题，尽管在这个例子中没有显式设置约束条件。
    jac=True,
    # 设置为True表示要求backprop函数计算梯度（损失函数关于权重的导数）。如果backprop函数能够返回梯度，这将加速优化过程，因为minimize可以利用梯度信息更有效地搜索最小值。
    options={"maxfun": 250},
    # 传递给优化算法的选项字典。{"maxfun": 250}指定了最大函数评估次数，即优化过程中允许的最大迭代次数。达到这个次数后，即使没有找到最优解，优化过程也会停止。
)
fmin
# fmin是minimize函数的返回值，它是一个OptimizeResult对象，包含了优化过程的信息，如最终找到的最小值（fun属性）、最优参数（x属性）、收敛状态（success属性）等。

 message: Max. number of function evaluations reached
 success: False
  status: 3
     fun: 0.32904608244443867
       x: [-1.022e+00 -2.175e-03 ... -7.950e-01 -1.487e+00]
     nit: 22
     jac: [ 2.058e-04 -4.351e-07 ...  1.801e-06 -1.422e-05]
    nfev: 250

由于目标函数不太可能完全收敛，我们对迭代次数进行了限制。 我们的总代价已经下降到0.5以下，这是算法正常工作的一个很好的指标。 让我们使用它发现的参数，并通过网络转发，以获得一些预测。

让我们使用它找到的参数，并通过网络前向传播以获得预测。

In [32]:
X = np.matrix(X)
theta1 = np.matrix(
    np.reshape(
        fmin.x[: hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))
    )
)
theta2 = np.matrix(
    np.reshape(
        fmin.x[hidden_size * (input_size + 1) :], (num_labels, (hidden_size + 1))
    )
)

a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
# h是模型的预测输出，是每个样本的类别概率分布。

y_pred = np.array(np.argmax(h, axis=1) + 1)
# np.argmax(h, axis=1)：找到每个样本在概率向量h中的最大值索引，这代表了预测的类别。axis=1表示按行（样本）进行操作。
# + 1：因为argmax返回的是从0开始的索引，但通常类别标签是从1开始的，所以加上1来得到正确的标签值。
# 最终，y_pred是模型对输入数据X的预测结果，一个数组，包含每个样本的预测类别。

y_pred

array([[10],
       [10],
       [10],
       ...,
       [ 9],
       [ 9],
       [ 9]])

最后，我们可以计算准确度，看看我们训练完毕的神经网络效果怎么样。

In [33]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
accuracy = sum(map(int, correct)) / float(len(correct))
print("accuracy = {0}%".format(accuracy * 100))

accuracy = 99.38%


我们已经成功地实施了一个基本的反向传播神经网络，并用它来分类手写数字图像。 在下一个练习中，我们将介绍另一个强大的监督学习算法，支持向量机。